In [1]:
%env BASE_DIR=/home/jupyter/snippets

env: BASE_DIR=/home/jupyter/snippets


In [2]:
!gsutil cp gs://cs327e-open-access/fs_shopify.zip $BASE_DIR

Copying gs://cs327e-open-access/fs_shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [3]:
!gsutil cp gs://cs327e-open-access/fs_samples.zip $BASE_DIR

Copying gs://cs327e-open-access/fs_samples.zip...
/ [1 files][  4.5 KiB/  4.5 KiB]                                                
Operation completed over 1 objects/4.5 KiB.                                      


In [4]:
!unzip -o $BASE_DIR/fs_shopify.zip

Archive:  /home/jupyter/snippets/fs_shopify.zip
   creating: fs_shopify/
  inflating: fs_shopify/apps.csv     
  inflating: fs_shopify/reviews.csv  
  inflating: fs_shopify/apps_categories.csv  
  inflating: fs_shopify/key_benefits.csv  
  inflating: fs_shopify/categories.csv  
  inflating: fs_shopify/pricing_plans.csv  
  inflating: fs_shopify/pricing_plan_features.csv  


In [5]:
!unzip -o $BASE_DIR/fs_samples.zip

Archive:  /home/jupyter/snippets/fs_samples.zip
   creating: fs_samples/
  inflating: fs_samples/delete_single.py  
  inflating: fs_samples/write_batch.py  
  inflating: fs_samples/read_nested.py  
  inflating: fs_samples/read_count.py  
  inflating: fs_samples/write_single.py  
  inflating: fs_samples/read_where.py  
  inflating: fs_samples/write_nested.py  
  inflating: fs_samples/delete_field.py  
  inflating: fs_samples/read_single.py  
  inflating: fs_samples/update_single.py  
  inflating: fs_samples/read_doc_with_filter2.py  


In [ ]:
import pandas as pd
from google.cloud import firestore

db = firestore.Client()

def batch_commit(rows):
    batch = db.batch()
    for reviews_row in reviews_rows:
        reviews_record = {}
        #reviews_record['rid'] = 'RID_' + reviews_row[0]
        reviews_record['app_id'] = reviews_row[0]
        reviews_record['author'] = reviews_row[1]
        reviews_record['rating'] = reviews_row[2]
        reviews_record['posted_at'] = reviews_row[3]

        reviews_ref = db.collection('reviews').document(reviews_row[0])
        batch.set(reviews_ref, reviews_record)
        for apps_row in apps_rows:
            if apps_row[0] == reviews_row[0]:
                apps_record = {}
                apps_record['id'] = apps_row[0]
                apps_record['url'] = apps_row[1]
                apps_record['title'] = apps_row[2]
                apps_record['developer'] = apps_row[3]
                apps_record['developer_link'] = apps_row[4]
                apps_record['icon'] = apps_row[5]
                apps_record['reviews_count'] = apps_row[7]
                
                apps_ref = reviews_ref.collection('apps').document(apps_row[0])
                batch.set(apps_ref, apps_record)
                
    batch.commit()

df = pd.read_csv('fs_shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
reviews_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

# Split rows into mini-batches of size 200
chunk_size = 200
for i in range(0, len(reviews_rows), chunk_size):
    chunk = reviews_rows[i:i+chunk_size]
    batch_commit(chunk)

In [3]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pplan_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
pplanf_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
kb_rows = df.values.tolist()

for apps_row in apps_rows: 
    apps_record = {}
    apps_record['id'] = apps_row[0]
    apps_record['url'] = apps_row[1]
    apps_record['title'] = apps_row[2]
    apps_record['developer'] = apps_row[3]
    apps_record['developer_link'] = apps_row[4]
    apps_record['icon'] = apps_row[5]
    apps_record['rating'] = apps_row[6]
    apps_record['reviews_count'] = apps_row[7]
    
    apps_ref = db.collection('apps').document(apps_row[0])
    batch.set(apps_ref, apps_record)
    
    for pplan_row in pplan_rows:
        
        if apps_row[0] == pplan_row[1]:
            pplan_record = {}
            pplan_record['ppid'] = pplan_row[0]
            pplan_record['title'] = pplan_row[2]
            pplan_record['price'] = pplan_row[3]
            
    for pplanf_row in pplanf_rows:
        
        if apps_row[0] == pplanf_row[0]:
            pplan_record['feature'] = pplanf_row[0]
            
            pplan_ref = apps_ref.collection('pricing_plans').document(pplan_row[0])
            batch.set(pplan_ref, pplan_record)
    
    for kb_row in kb_rows:
        if apps_row[0] == kb_row[0]:
            kb_record = {}
            kb_record['title'] = kb_row[1] 
            kb_record['description'] = kb_row[2]
        
            kb_ref = apps_ref.collection('key_benefits').document(kb_row[0])
            batch.set(kb_ref, kb_record)
        
    
    batch.commit()

In [1]:
import pandas as pd
from google.cloud import firestore

db = firestore.Client()

df = pd.read_csv('fs_shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
ac_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/categories.csv', sep=',', header=0, lineterminator='\n')
c_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

batch = db.batch()
for c_row in c_rows:
    c_record = {}
    c_record['category_id'] = c_row[0]
    c_record['title'] = c_row[1]
        
    categories_ref = db.collection('categories').document(c_row[1])
    batch.set(categories_ref, c_record)
        
    for ac_row in ac_rows:
        if c_row[0] == ac_row[1]:
            for apps_row in apps_rows:
                if apps_row[0]==ac_row[0]:
                    apps_record = {}
                    apps_record['id'] = apps_row[0]
                    apps_record['url'] = apps_row[1]
                    apps_record['title'] = apps_row[2]
                    apps_record['developer'] = apps_row[3]
                    apps_record['developer_link'] = apps_row[4]
                    apps_record['icon'] = apps_row[5]
                    apps_record['ratings'] = apps_row[6]
                    apps_record['reviews_count'] = apps_row[7]
            
                    apps_ref = categories_ref.collection('apps').document(apps_row[0])
                    batch.set(apps_ref, apps_record)

                
    batch.commit()





In [ ]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter
from google.cloud.firestore_v1 import aggregation

db = firestore.Client()
apps_ref = db.collection('apps')
query = apps_ref.order_by('title', direction=firestore.Query.DESCENDING)
aggregate_query = aggregation.AggregationQuery(query)
aggregate_query.count()
results = aggregate_query.get()
    
for result in results:
    print(f"Count of documents in apps is: {result[0].value}")

    
db = firestore.Client()
categories_ref = db.collection('categories')
query = categories_ref.order_by('title', direction=firestore.Query.DESCENDING)
aggregate_query = aggregation.AggregationQuery(query)
aggregate_query.count()
results = aggregate_query.get()
    
for result in results:
    print(f"Count of documents in categories is: {result[0].value}")
    

results=db.collection('categories')
results=results.stream()
for result in results:
    apps_ref = db.collection('categories').document(result.id).collection('apps')
    query = apps_ref.order_by('id', direction=firestore.Query.DESCENDING)
    aggregate_query = aggregation.AggregationQuery(query)
    aggregate_query.count()
    results = aggregate_query.get()
    
    for result in results:
        print(f" In the subcollection, apps which is in the collection categories, the count is: {result[0].value}")
        
results=db.collection('apps')
results=results.stream()
for result in results:
    apps_ref = db.collection('apps').document(result.id).collection('key_benefits')
    query = apps_ref.order_by('title', direction=firestore.Query.DESCENDING)
    aggregate_query = aggregation.AggregationQuery(query)
    aggregate_query.count()
    results = aggregate_query.get()
    
    for result in results:
        print(f" In the subcollection, key_benefits which is in the collection apps, the count is: {result[0].value}")
        
results=db.collection('apps')
results=results.stream()
for result in results:
    apps_ref = db.collection('apps').document(result.id).collection('pricing_plans')
    query = apps_ref.order_by('title', direction=firestore.Query.DESCENDING)
    aggregate_query = aggregation.AggregationQuery(query)
    aggregate_query.count()
    results = aggregate_query.get()
    
    for result in results:
        print(f" In the subcollection, pricing_plans which is in the collection apps, the count is: {result[0].value}")

Count of documents in apps is: 3549
Count of documents in categories is: 12
 In the subcollection, apps which is in the collection categories, the count is: 310
 In the subcollection, apps which is in the collection categories, the count is: 133
 In the subcollection, apps which is in the collection categories, the count is: 176
 In the subcollection, apps which is in the collection categories, the count is: 251
 In the subcollection, apps which is in the collection categories, the count is: 794
 In the subcollection, apps which is in the collection categories, the count is: 602
 In the subcollection, apps which is in the collection categories, the count is: 83
 In the subcollection, apps which is in the collection categories, the count is: 190
 In the subcollection, apps which is in the collection categories, the count is: 249
 In the subcollection, apps which is in the collection categories, the count is: 957
 In the subcollection, apps which is in the collection categories, the coun

In [60]:
from google.cloud.firestore_v1.base_query import FieldFilter
db = firestore.Client()
categories_ref = db.collection('categories').document('Productivity')
apps_ref=categories_ref.collection('apps')
query = apps_ref.order_by('ratings', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()

for result in results:
    print(result.get('id'),'-> title:',result.get('title'),'developer:',result.get('developer'),'ratings:',result.get('ratings'),'reviews:',result.get('reviews_count'))


fe5ae45c-379c-42bf-be7d-b5f6e15fc13c -> title: Customer Tags developer: Union Works Apps ratings: 5.0 reviews: 21
f99bb1e3-f326-4f10-8901-491652e9809b -> title: Order Tagger developer: Union Works Apps ratings: 5.0 reviews: 68
f864e3bd-da0e-41dc-be65-984325331475 -> title: SilkRoad ‑ Facebook Auto Ads developer: SilkRoad ratings: 5.0 reviews: 2
f5344e64-9cda-4d97-b198-2aaeb5170518 -> title: Xporter Data Export Tool developer: Modd Apps Inc. ratings: 5.0 reviews: 223
ed77a32d-0fa3-458b-b639-e01ea7b78ec0 -> title: FraudBlock Fraud Prevention developer: ShopFox ratings: 5.0 reviews: 6
e0e231d6-4988-4a8c-ad84-cf18d0f38738 -> title: Automation Fox developer: Automation Fox ratings: 5.0 reviews: 1
d75fa395-3a4d-41c3-88d1-2f802c1e2411 -> title: AirPower developer: BaseGenius ratings: 5.0 reviews: 1
d0be03ff-f74d-4fb5-8d2c-79541a1aea1b -> title: Excelify developer: Excelify.io ratings: 5.0 reviews: 178
c9a86f8c-817e-4bb5-bbdd-e0e669bd7816 -> title: UPC Code Manager developer: Ventures & Advent

In [62]:
from google.cloud.firestore_v1.base_query import FieldFilter
db = firestore.Client()
apps_ref = db.collection('apps')
query = apps_ref.order_by('reviews_count', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()

for result in results:
    print(result.get('id'),'-> title:',result.get('title'),'developer:',result.get('developer'),'rating:',result.get('rating'),'reviews:',result.get('reviews_count'))


d9f142ee-b141-4dc4-9353-173db61d2eb0 -> title: Privy ‑ Exit Pop Ups & Email developer: Privy rating: 4.7 reviews: 23078
78ea0810-c008-4a4e-a82f-de0c790e3286 -> title: Free Shipping Bar developer: Hextom rating: 4.9 reviews: 8737
b88488b0-9912-44d3-b736-224c36f09d95 -> title: Sales Pop ‑ Popup Notification developer: CartKit rating: 4.8 reviews: 6905
e528a60e-94f8-4e92-80e2-5bc6013b8283 -> title: BEST Currency Converter developer: Grizzly Apps rating: 4.8 reviews: 5986
be2640c4-01b5-4d52-9f68-cae8c0734d0d -> title: Recart FB Messenger Marketing developer: Recart rating: 4.8 reviews: 5596
70bff9e0-4316-4cc6-84ce-92fcd1bc6925 -> title: EU Cookie Bar ‑ Cookie GDPR developer: Booster Apps rating: 4.7 reviews: 5259
171816e2-27d4-4552-a65e-ab44a312fe04 -> title: Sales Pop Master ‑ Countdown developer: Autoketing rating: 4.8 reviews: 4931
9025eff0-d714-4df1-930f-43f5582979ad -> title: Ultimate Sales Boost developer: Hextom rating: 4.8 reviews: 4847
404183ac-6d52-4960-85d2-8350f37c2f0b -> title